In [1]:
import os
import keras
import pprint
import numpy as np
import pickle
import time
# import dill as pickle
import matplotlib.pyplot as plt
from keras import applications
from keras import optimizers
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
from PIL import Image
%matplotlib inline

traindir = '/home/saniea/imagedata/cifar10/'
valdir = '/home/saniea/imagedata/cifar10/test_batch'

Using TensorFlow backend.


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def convert(data):
    # magic recursive function
    if isinstance(data, bytes):      return data.decode('ascii')
    if isinstance(data, dict):       return dict(map(convert, data.items()))
    if isinstance(data, tuple):      return map(convert, data)
    if isinstance(data, list):       return list(map(convert, data))
    return data

# TODO: function to print image if needed
# X = X.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
# # X = X.reshape(50000, 3, 32, 32)
# # X = X.transpose(0, 2, 3, 1)
# plt.imshow(X[5], interpolation='none')
# print(label_names[int(Y[5])])
# plt.figure()
# plt.imshow(X[7])
# print(label_names[int(Y[7])])

In [3]:
for root, dirs, filenames in os.walk(traindir):
    files = filenames
# print(files)

data = [] # list of all training data files to be unpacked, total 50k images
[data.append(file) for file in files if (file.startswith('data') == True)]
data.sort()
# print(data)

test = unpickle(str(traindir+'data_batch_1'))
# print(test.keys())

X = np.empty((50000,3072))
Y = np.empty((50000))
batch_len = 10000

for i, batch in enumerate (data):
    batch = unpickle(str(traindir+batch))
    current_batch = {k.decode() : v for k, v in batch.items()}
#     batch_data = current_batch[b'data']
#     batch_labels = current_batch[b'labels']
#     j = i + 1
    X[batch_len*i:batch_len*(i+1)] = current_batch['data']
    Y[batch_len*i:batch_len*(i+1)] = current_batch['labels']
    
    # i = 0, index(j) = i + 1 = 1 --> 00000 - 10000
    # i = 1, index(j) = i + 1 = 2 --> 10000 - 20000
    # i = 2, index(j) = i + 1 = 3 --> 20000 - 30000

meta_b = unpickle(str(traindir+'batches.meta'))
meta = {k.decode() : v for k, v in meta_b.items()}
label_names = meta['label_names']

# test_batch = unpickle(valdir)
val_batch = {k.decode():v for k, v in unpickle(valdir).items()}
X_val = val_batch['data']
Y_val = val_batch['labels']
    
print('Training data size: ' + str(X.shape) + '\n'
     +'Training labels size: ' + str(len(Y)) + '\n'
     +'Validation data size: ' + str(X_val.shape) + '\n'
     +'Validation labels size: ' + str(len(Y_val)) + '\n')


Training data size: (50000, 3072)
Training labels size: 50000
Validation data size: (10000, 3072)
Validation labels size: 10000

